In [1]:
# libraries 
import pandas as pd 
import os
import numpy as np
import time
import random
from csv import reader
import time

# plots:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


# scraping
from bs4 import BeautifulSoup
from lxml import etree #for using XPATH with beautifulsoup
import requests
import numpy as np

#JSON
import json

# GeoJSON
# import geopandas as gpd

# regular expression
import re 

# concurrent futures - boosts the process of scraping utilicing the CPU better
# two main classes- 
#       1) executor class: manages all the threads and workload
#       2) futures class: creates a little instance and manages data coming back
import concurrent.futures

# libraries: 
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import glob 

# ---------------------------------------------------------------------------------------------------

# two parts : 

## part 1: Expand columns 
## part 2: filter out columns of no use

# ---------------------------------------------------------------------------------------------------

# ------------ read in the data 

In [2]:
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_4_Merge_files\Boliga_merged_files.csv'
database = pd.read_csv(path,encoding='utf-8', low_memory=False)

In [3]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031595 entries, 0 to 1031594
Data columns (total 48 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   unitId                  1031595 non-null  object 
 1   isSalesValid            1031595 non-null  bool   
 2   isActive                1031595 non-null  bool   
 3   addressString           1031595 non-null  object 
 4   gadeString              1031595 non-null  object 
 5   hus_nr                  1031595 non-null  object 
 6   etagebetegn             270092 non-null   object 
 7   side_Doernr             215293 non-null   object 
 8   zipString               1031595 non-null  object 
 9   zipCodeString           1031595 non-null  int64  
 10  imageUrl                417998 non-null   object 
 11  isApartment             1031595 non-null  bool   
 12  propertyType            1031595 non-null  int64  
 13  municipalityCode        1031595 non-null  int64  
 14  es

In [ ]:
# database.describe()

# ----------------- Drop un-useful columns 

### they are : 
    * imageUrl                
    * isBlocked
    * isInSalesAgent	
    * isInWatchlist
    * watchlistNote
    * canGetVR
    * relatedImages
    * ownerShipInfo           
    * propDataSettings        
    * propDataSettingTypes    
    * boligAfgift             
    * agentInfo      
    * soldAgentInfo           
    


In [4]:
drop_columns = [
    'imageUrl',
    'isSalesValid',
    'isBlocked',
    'isInSalesAgent',
    'isInWatchlist',
    'watchlistNote',
    'canGetVR',
    'relatedImages',
    'ownerShipInfo',           
    'propDataSettings',        
    'propDataSettingTypes',    
    'boligAfgift',             
    'agentInfo',      
    'soldAgentInfo',
    'esrOwnershipInfo'        
]
database = database.drop(columns= drop_columns)


In [5]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031595 entries, 0 to 1031594
Data columns (total 33 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   unitId                  1031595 non-null  object 
 1   isActive                1031595 non-null  bool   
 2   addressString           1031595 non-null  object 
 3   gadeString              1031595 non-null  object 
 4   hus_nr                  1031595 non-null  object 
 5   etagebetegn             270092 non-null   object 
 6   side_Doernr             215293 non-null   object 
 7   zipString               1031595 non-null  object 
 8   zipCodeString           1031595 non-null  int64  
 9   isApartment             1031595 non-null  bool   
 10  propertyType            1031595 non-null  int64  
 11  municipalityCode        1031595 non-null  int64  
 12  esrCode                 1031595 non-null  int64  
 13  lat                     1031595 non-null  float64
 14  lo

# ------------ change column names 

In [6]:
# Create a dictionary to map old column names to new column names
column_name_mapping = {
    'unitId': 'guid_code', 
    'isActive': 'is_on_sale',   
    'addressString': 'addressString', 
    'gadeString': 'streetname', 
    'hus_nr': 'address_number', 
    'etagebetegn': 'address_floor_number', 
    'side_Doernr': 'address_door_number', 
    'zipString': 'Zip_code_name', 
    'zipCodeString': 'Zip_code',  
    'isApartment': 'isApartment',   
    'propertyType': 'propertyType_code',   
    'municipalityCode': 'municipalityCode',  
    'esrCode': 'property_esrCode',  
    'lat': 'latitude',
    'lon': 'longitude',
    'isCoop': 'is_cooperative_property',   
    'addressFloor': 'addressFloor', 
    'ejendomsværdiskat': 'ejendomsværdiskat',
    'grundskyld': 'grundskyld',
    'propertyTypeName': 'propertyType_Name', 
    'bbrInfoBox': 'char_bbrInfoBox', 
    'buildingInfo': 'char_buildingInfo', 
    'unitInfo': 'char_unitInfo', 
    'lotInfo': 'char_lotInfo', 
    'cadastralInfos': 'char_cadastralInfos', 
    'additionalBuldingInfos': 'char_additionalBuldingInfos', 
    'floorInfos': 'char_floorInfos', 
    'additionalFloorInfos': 'char_additionalFloorInfos', 
    'salesInfos': 'Price_Transaction_sales', 
    'evaluationInfos': 'Price_Public_evaluations', 
    'currentEstate': 'Price_current_listing', 
    'previousEstates': 'Price_previous_listing', 
    'bfenr': 'bfenr'  
    }

# Rename columns using the rename method
database = database.rename(columns=column_name_mapping)

In [7]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031595 entries, 0 to 1031594
Data columns (total 33 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   guid_code                    1031595 non-null  object 
 1   is_on_sale                   1031595 non-null  bool   
 2   addressString                1031595 non-null  object 
 3   streetname                   1031595 non-null  object 
 4   address_number               1031595 non-null  object 
 5   address_floor_number         270092 non-null   object 
 6   address_door_number          215293 non-null   object 
 7   Zip_code_name                1031595 non-null  object 
 8   Zip_code                     1031595 non-null  int64  
 9   isApartment                  1031595 non-null  bool   
 10  propertyType_code            1031595 non-null  int64  
 11  municipalityCode             1031595 non-null  int64  
 12  property_esrCode             1031595 non-n

# ----------------- change column order 

In [9]:
# Specify the desired column order
new_column_order = [
    'guid_code',
    'property_esrCode',
    'municipalityCode',  
    'addressString', 
    'streetname', 
    'address_number', 
    'address_floor_number',
    'addressFloor', 
    'address_door_number', 
    'Zip_code_name', 
    'Zip_code',      
    'is_on_sale',   
    'propertyType_Name',
    'propertyType_code',  
    'isApartment',
    'is_cooperative_property',
    'char_bbrInfoBox', 
    'char_buildingInfo', 
    'char_unitInfo', 
    'char_lotInfo', 
    'char_cadastralInfos', 
    'char_additionalBuldingInfos', 
    'char_floorInfos', 
    'char_additionalFloorInfos',
    'Price_Transaction_sales', 
    'Price_Public_evaluations', 
    'Price_current_listing', 
    'Price_previous_listing',
    'latitude',
    'longitude',
    'bfenr',
    'ejendomsværdiskat',
    'grundskyld'
]

# Reorder columns
database = database[new_column_order]


In [10]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031595 entries, 0 to 1031594
Data columns (total 33 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   guid_code                    1031595 non-null  object 
 1   property_esrCode             1031595 non-null  int64  
 2   municipalityCode             1031595 non-null  int64  
 3   addressString                1031595 non-null  object 
 4   streetname                   1031595 non-null  object 
 5   address_number               1031595 non-null  object 
 6   address_floor_number         270092 non-null   object 
 7   addressFloor                 270092 non-null   object 
 8   address_door_number          215293 non-null   object 
 9   Zip_code_name                1031595 non-null  object 
 10  Zip_code                     1031595 non-null  int64  
 11  is_on_sale                   1031595 non-null  bool   
 12  propertyType_Name            1031595 non-n

# ------- Save the outcome as is 

In [33]:
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\Boliga_nameChanged_columnsDeleted.csv'
database.to_csv(path,encoding='utf-8')

In [18]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031595 entries, 0 to 1031594
Data columns (total 34 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   Unnamed: 0                   1031595 non-null  int64  
 1   guid_code                    1031595 non-null  object 
 2   property_esrCode             1031595 non-null  int64  
 3   municipalityCode             1031595 non-null  int64  
 4   addressString                1031595 non-null  object 
 5   streetname                   1031595 non-null  object 
 6   address_number               1031595 non-null  object 
 7   address_floor_number         270092 non-null   object 
 8   addressFloor                 270092 non-null   object 
 9   address_door_number          215293 non-null   object 
 10  Zip_code_name                1031595 non-null  object 
 11  Zip_code                     1031595 non-null  int64  
 12  is_on_sale                   1031595 non-n

# ------------------------- Analysing char_* columns --------------------------

# those columns are char_* columns (property characteristics) 
 
that is: 
   * char_bbrInfoBox              
   * char_buildingInfo            
   * char_unitInfo                
   * char_lotInfo                 
   * char_cadastralInfos          
   * char_additionalBuldingInfos  
   * char_floorInfos              
   * char_additionalFloorInfos

# ------- Analysing char_bbrInfoBox 

In [ ]:
database['char_bbrInfoBox'].tolist()

In [ ]:
import ast

# merged_data_bbrInfoBox = pd.DataFrame()

# for i in database['char_bbrInfoBox'].tolist():
#     row = ast.literal_eval(i)
#     df = pd.DataFrame([row])
    
#     # merge the df to the main dataframe
#     merged_data_bbrInfoBox = pd.concat([merged_data_bbrInfoBox, df], ignore_index=True)
    

# Collect DataFrames in a list
dfs = []
count = 1
for i in database['char_bbrInfoBox'].tolist():
    row = ast.literal_eval(i)
    df = pd.DataFrame([row])
    dfs.append(df)
    print(f'--- row nr. {count}')
    count+=1
# Concatenate the list of DataFrames
merged_data_bbrInfoBox = pd.concat(dfs, ignore_index=True)

In [32]:
merged_data_bbrInfoBox

,monthlyPropertyTax,lotSize,roomsQuantity,area,areaRecorded,evaluationPrice,basementSize
0,1604.0,0,5,141,0,2250000,0
1,2931.0,217,4,122,0,2500000,36
2,448.0,343,3,70,0,530000,0
3,1049.0,0,2,71,64,1550000,0
4,1272.0,103,3,86,0,960000,0
...,...,...,...,...,...,...,...
1031590,741.0,1084,8,246,0,720000,0
1031591,709.0,1023,4,144,0,670000,0
1031592,1250.0,750,5,119,0,800000,83
1031593,1266.0,6735,4,106,0,6600000,40


# ------- Analysing char_buildingInfo            

In [3]:
import ast

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'
data = pd.read_csv(path,encoding='utf-8')

# for i in data['char_buildingInfo'].tolist():
#     row = ast.literal_eval(i)
#     df = pd.DataFrame([row])
#     dfs.append(df)
#     print(f'--- row nr. {count}')
#     count+=1
# # Concatenate the list of DataFrames
# merged_data_bbrInfoBox = pd.concat(dfs, ignore_index=True)

C:\Users\ABC\AppData\Local\Temp\ipykernel_16064\3821745156.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path,encoding='utf-8')


In [7]:

keys_data = []
count = 1

# Sample list of strings
data_list = data['char_buildingInfo'].tolist()

# Extract keys using list comprehension and extend directly
keys_data = [key for row in map(ast.literal_eval, data_list) for key in row.keys()]



# --------------------------------------------- excplicit loop : same as above here
# for i in data['char_buildingInfo'].tolist():
#     row = ast.literal_eval(i)
#     keys = list(row.keys())
         
#     keys_data.extend(keys)
#     print(f'--- row nr. {count}')
#     count+=1



In [10]:
for i in set(keys_data):
    print(i)

roof
closedCoveredArea
openCoveredArea
outerWalls
asbestosMaterial
additionalHeatingSupplyCode
conservationCode
additionalOuterWallMaterial
objectStatus
secureRoomArea
appartmentsWithKitchensQuantity
buildingArea
accessAreaArea
buildInCarportArea
drainCode
heatingCode
notes
livingArea
coveredArea
floorDeviation
conservatoryArea
businessArea
heatingSupplyCode
drainPermission
conservationWorthy
constructionYear
buildingUsage
buildInOuthouseArea
waterSupplyCode
buildingNumber
additionalRoofMaterial
floorQuantity
latesstRebuildYear
carbageRoomArea
buildInGarageArea
appartmentsWithoutKitchensQuantity
otherArea


# ------- Analysing char_unitInfo 

In [20]:

keys_data = []
count = 1

# Sample list of strings
data_list = database['char_unitInfo'].tolist()

# Extract keys using list comprehension and extend directly
keys_data = [key for row in map(ast.literal_eval, data_list) for key in row.keys()]

for i in set(keys_data):
    print(i)


appliance
roomsBusinessQuantity
energyCode
areaLiving
areaClosedCovered
areaOpenCovered
kitchenCode
roomsQuantity
propertyCondemned
toiletCode
areaSocial
heatingCode
areaOther
notes
propertyUnitType
areaBusiness
areaUnit
bathroomCode
heatingSecondaryCode
bathroomQuantity
usageCode
heatingInstallationCode
toiletQuantity


In [ ]:
# import ast

# # merged_data_bbrInfoBox = pd.DataFrame()

# # for i in database['char_bbrInfoBox'].tolist():
# #     row = ast.literal_eval(i)
# #     df = pd.DataFrame([row])
    
# #     # merge the df to the main dataframe
# #     merged_data_bbrInfoBox = pd.concat([merged_data_bbrInfoBox, df], ignore_index=True)
    

# # Collect DataFrames in a list
# dfs = []
# count = 1
# for i in database['char_bbrInfoBox'].tolist():
#     row = ast.literal_eval(i)
#     df = pd.DataFrame([row])
#     dfs.append(df)
#     print(f'--- row nr. {count}')
#     count+=1
# # Concatenate the list of DataFrames
# merged_data_bbrInfoBox = pd.concat(dfs, ignore_index=True)

# ------- Analysing char_lotinfo

In [1]:
1031595/10000

103.1595

In [ ]:
import ast

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

# Specify the chunk size (number of rows to read at a time)
chunk_size = 10000

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['char_lotInfo'],chunksize=chunk_size)

keys_database = []

count = 0 
for chunk in csv_reader:
    count +=1
    print(f'----- chunk number {count} our of 104')
    
    # Sample list of strings
    data_list = chunk['char_lotInfo'].tolist()

    # Remove nan values from data_list
    non_empty_data_list = [str(obs) for obs in data_list if pd.notna(obs)]

    # Extract keys using list comprehension and extend directly
    keys_data = [key for row in map(ast.literal_eval, non_empty_data_list) for key in row.keys()]
    
    # add to they keys database
    keys_database.extend(keys_data)

    

In [11]:
set(keys_database)

{'drain', 'drainPermission', 'notes', 'watersupplyCode'}

# ------- Analysing char_cadastralinfos

In [ ]:
import ast

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

# Specify the chunk size (number of rows to read at a time)
chunk_size = 10000

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['char_cadastralInfos'],chunksize=chunk_size)



#  --------------------------------- old code

keys_database = []
number_of_objects = []

count = 0 
stoppa = 0
for chunk in csv_reader:
    count +=1
    stoppa +=1
    print(f'----- chunk number {count} our of 104')
    
    # Sample list of strings
    data_list = chunk['char_cadastralInfos'].tolist()

    # Remove nan values from data_list
    non_empty_data_list = [str(obs) for obs in data_list if pd.notna(obs) and obs != '']

    # Extract keys using list comprehension and extend directly
#     keys_data = [key for row in map(ast.literal_eval, non_empty_data_list) for key in row.keys()]
    keys_data = []

    # Extract keys using a normal for loop
    for row_str in non_empty_data_list:
        row = ast.literal_eval(row_str)
#         display(row)
#         print(f'number of objects in row : {len(row)}')

        # add number of objects in each row of the column to see it there are more than 1
        number_of_objects.append(len(row))
        for row_item in row:
            for key in row_item.keys():
                keys_data.append(key)

    # add to they keys database
    keys_database.extend(keys_data)

    

In [37]:
for i in set(keys_database):
    print(i)

primaryCadestral
munipalOwnerAssociationName
countryOwnerAssociationCode
area
buildingNumber
roadArea
countryOwnerAssociationName
munipalOwnerAssociationCode
cadestralNumber


# ------- Analysing char_additionalBuildinginfos

In [ ]:
import ast

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

# Specify the chunk size (number of rows to read at a time)
chunk_size = 10000

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['char_additionalBuldingInfos'],chunksize=chunk_size)



#  --------------------------------- old code

keys_database = []
number_of_objects = []

count = 0 
stoppa = 0
for chunk in csv_reader:
    count +=1
    stoppa +=1
    print(f'----- chunk number {count} our of 104')
    
    # Sample list of strings
    data_list = chunk['char_additionalBuldingInfos'].tolist()

    # Remove nan values from data_list
    non_empty_data_list = [str(obs) for obs in data_list if pd.notna(obs) and obs != '']

    # Extract keys using list comprehension and extend directly
#     keys_data = [key for row in map(ast.literal_eval, non_empty_data_list) for key in row.keys()]
    keys_data = []

    # Extract keys using a normal for loop
    for row_str in non_empty_data_list:
        row = ast.literal_eval(row_str)
#         display(row)
#         print(f'number of objects in row : {len(row)}')

        # add number of objects in each row of the column to see it there are more than 1
        number_of_objects.append(len(row))
        for row_item in row:
            for key in row_item.keys():
                keys_data.append(key)

    # add to they keys database
    keys_database.extend(keys_data)

    
for i in set(keys_database):
    print(i)

In [ ]:
set(number_of_objects)

# ------- Analysing char_floorInfos

In [ ]:
import ast

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

# Specify the chunk size (number of rows to read at a time)
chunk_size = 10000

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['char_floorInfos'],chunksize=chunk_size)



#  --------------------------------- old code

keys_database = []
number_of_objects = []

count = 0 
stoppa = 0
for chunk in csv_reader:
    count +=1
    stoppa +=1
    print(f'----- chunk number {count} our of 104')
    
    # Sample list of strings
    data_list = chunk['char_floorInfos'].tolist()

    # Remove nan values from data_list
    non_empty_data_list = [str(obs) for obs in data_list if pd.notna(obs) and obs != '']

    # Extract keys using list comprehension and extend directly
#     keys_data = [key for row in map(ast.literal_eval, non_empty_data_list) for key in row.keys()]
    keys_data = []

    # Extract keys using a normal for loop
    for row_str in non_empty_data_list:
        row = ast.literal_eval(row_str)
#         display(row)
#         print(f'number of objects in row : {len(row)}')

        # add number of objects in each row of the column to see it there are more than 1
        number_of_objects.append(len(row))
        for row_item in row:
            for key in row_item.keys():
                keys_data.append(key)

    # add to they keys database
    keys_database.extend(keys_data)



In [44]:
for i in set(keys_database):
    print(i)

designation
accessArea
area
usedAtticArea
notes
highBasementArea
basementLivingArea


In [41]:
set(number_of_objects)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19}

In [43]:
from collections import Counter

# Count unique values
value_counts = Counter(keys_database)
value_counts

Counter({'designation': 888208,
         'area': 888208,
         'usedAtticArea': 888208,
         'basementLivingArea': 888208,
         'highBasementArea': 888208,
         'accessArea': 888208,
         'notes': 888208})

# ------- Analysing char_additionalFloorInfos

In [ ]:
import ast

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

# Specify the chunk size (number of rows to read at a time)
chunk_size = 10000

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['char_additionalFloorInfos'],chunksize=chunk_size)



#  --------------------------------- old code

keys_database = []
number_of_objects = []

count = 0 
stoppa = 0
for chunk in csv_reader:
    count +=1
    stoppa +=1
    print(f'----- chunk number {count} our of 104')
    
    # Sample list of strings
    data_list = chunk['char_additionalFloorInfos'].tolist()

    # Remove nan values from data_list
    non_empty_data_list = [str(obs) for obs in data_list if pd.notna(obs) and obs != '']

    # Extract keys using list comprehension and extend directly
#     keys_data = [key for row in map(ast.literal_eval, non_empty_data_list) for key in row.keys()]
    keys_data = []

    # Extract keys using a normal for loop
    for row_str in non_empty_data_list:
        row = ast.literal_eval(row_str)
#         display(row)
#         print(f'number of objects in row : {len(row)}')

        # add number of objects in each row of the column to see it there are more than 1
        number_of_objects.append(len(row))
        for row_item in row:
            for key in row_item.keys():
                keys_data.append(key)

    # add to they keys database
    keys_database.extend(keys_data)

In [46]:
for i in set(keys_database):
    print(i)

designation
accessArea
area
usedAtticArea
notes
highBasementArea
basementLivingArea


In [ ]:
set(number_of_objects)

In [48]:
from collections import Counter

# Count unique values
value_counts = Counter(keys_database)
value_counts

Counter({'designation': 207881,
         'area': 207881,
         'usedAtticArea': 207881,
         'basementLivingArea': 207881,
         'highBasementArea': 207881,
         'accessArea': 207881,
         'notes': 207881})

# ----------------------- Expanding column: bbrInfoBox
## i will take two columns guid, municipality code, property code and bbrInfoBox - expand the bbrInfoBox column and save it as a seperated file

In [3]:
import ast

#---------------------  read in the big file for only the key columns here

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['guid_code', 'property_esrCode','municipalityCode', 'char_bbrInfoBox'])


In [4]:
csv_reader

,guid_code,property_esrCode,municipalityCode,char_bbrInfoBox
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"{'monthlyPropertyTax': 1604.0, 'lotSize': 0, '..."
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"{'monthlyPropertyTax': 2931.0, 'lotSize': 217,..."
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"{'monthlyPropertyTax': 448.0, 'lotSize': 343, ..."
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,"{'monthlyPropertyTax': 1049.0, 'lotSize': 0, '..."
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"{'monthlyPropertyTax': 1272.0, 'lotSize': 103,..."
...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"{'monthlyPropertyTax': 741.0, 'lotSize': 1084,..."
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"{'monthlyPropertyTax': 709.0, 'lotSize': 1023,..."
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"{'monthlyPropertyTax': 1250.0, 'lotSize': 750,..."
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"{'monthlyPropertyTax': 1266.0, 'lotSize': 6735..."


In [6]:
# Convert strings to dictionaries
csv_reader['char_bbrInfoBox_dic'] = csv_reader['char_bbrInfoBox'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else {})

In [10]:
# Normalize the dictionaries into separate columns
df_expanded = pd.json_normalize(csv_reader['char_bbrInfoBox_dic'])


In [12]:
# Concatenate the expanded DataFrame with the original DataFrame
csv_reader = pd.concat([csv_reader, df_expanded], axis=1)
csv_reader

,guid_code,property_esrCode,municipalityCode,char_bbrInfoBox,char_bbrInfoBox_dic,monthlyPropertyTax,lotSize,roomsQuantity,area,areaRecorded,evaluationPrice,basementSize
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"{'monthlyPropertyTax': 1604.0, 'lotSize': 0, '...","{'monthlyPropertyTax': 1604.0, 'lotSize': 0, '...",1604.0,0,5,141,0,2250000,0
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"{'monthlyPropertyTax': 2931.0, 'lotSize': 217,...","{'monthlyPropertyTax': 2931.0, 'lotSize': 217,...",2931.0,217,4,122,0,2500000,36
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"{'monthlyPropertyTax': 448.0, 'lotSize': 343, ...","{'monthlyPropertyTax': 448.0, 'lotSize': 343, ...",448.0,343,3,70,0,530000,0
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,"{'monthlyPropertyTax': 1049.0, 'lotSize': 0, '...","{'monthlyPropertyTax': 1049.0, 'lotSize': 0, '...",1049.0,0,2,71,64,1550000,0
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"{'monthlyPropertyTax': 1272.0, 'lotSize': 103,...","{'monthlyPropertyTax': 1272.0, 'lotSize': 103,...",1272.0,103,3,86,0,960000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"{'monthlyPropertyTax': 741.0, 'lotSize': 1084,...","{'monthlyPropertyTax': 741.0, 'lotSize': 1084,...",741.0,1084,8,246,0,720000,0
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"{'monthlyPropertyTax': 709.0, 'lotSize': 1023,...","{'monthlyPropertyTax': 709.0, 'lotSize': 1023,...",709.0,1023,4,144,0,670000,0
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"{'monthlyPropertyTax': 1250.0, 'lotSize': 750,...","{'monthlyPropertyTax': 1250.0, 'lotSize': 750,...",1250.0,750,5,119,0,800000,83
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"{'monthlyPropertyTax': 1266.0, 'lotSize': 6735...","{'monthlyPropertyTax': 1266.0, 'lotSize': 6735...",1266.0,6735,4,106,0,6600000,40


In [13]:
########################################################################################################
# -------------------  save the result !       -------------------------
########################################################################################################
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\Bolgia_char_bbrInfoBox_Expanded.csv'
csv_reader.to_csv(path, encoding='utf-8')


# ----------------------- Expanding column: char_buildingInfo
## i will take two columns guid, municipality code, property code and bbrInfoBox - expand the bbrInfoBox column and save it as a seperated file

In [2]:
import ast

#---------------------  read in the big file for only the key columns here

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['guid_code', 'property_esrCode','municipalityCode', 'char_buildingInfo'])
csv_reader

,guid_code,property_esrCode,municipalityCode,char_buildingInfo
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"{'buildingNumber': 1, 'buildingUsage': 'Række-..."
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"{'buildingNumber': 1, 'buildingUsage': 'Række-..."
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"{'buildingNumber': 1, 'buildingUsage': 'Dobbel..."
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,"{'buildingNumber': 1, 'buildingUsage': 'Etageb..."
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"{'buildingNumber': 1, 'buildingUsage': 'Række-..."
...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"{'buildingNumber': 1, 'buildingUsage': 'Fritli..."
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"{'buildingNumber': 1, 'buildingUsage': 'Fritli..."
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"{'buildingNumber': 1, 'buildingUsage': 'Fritli..."
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"{'buildingNumber': 4, 'buildingUsage': 'Fritli..."


In [3]:
# Convert strings to dictionaries
csv_reader['char_buildingInfo_dic'] = csv_reader['char_buildingInfo'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else {})

In [6]:
# Normalize the dictionaries into separate columns
df_expanded = pd.json_normalize(csv_reader['char_buildingInfo_dic'])

# Concatenate the expanded DataFrame with the original DataFrame
csv_reader = pd.concat([csv_reader, df_expanded], axis=1)
csv_reader

,guid_code,property_esrCode,municipalityCode,char_buildingInfo,char_buildingInfo_dic,buildingNumber,buildingUsage,constructionYear,floorQuantity,latesstRebuildYear,...,additionalHeatingSupplyCode,additionalOuterWallMaterial,additionalRoofMaterial,asbestosMaterial,objectStatus,floorDeviation,appartmentsWithKitchensQuantity,appartmentsWithoutKitchensQuantity,secureRoomArea,notes
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"{'buildingNumber': 1, 'buildingUsage': 'Række-...","{'buildingNumber': 1, 'buildingUsage': 'Række-...",1,"Række-, kæde-, eller dobbelthus (lodret adskil...",2008,1,0,...,(UDFASES) Bygningen har ingen supplerende varme,-,-,-,Bygning - stamdata,Bygningen har ikke afvigende etager,1,0,0,[]
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"{'buildingNumber': 1, 'buildingUsage': 'Række-...","{'buildingNumber': 1, 'buildingUsage': 'Række-...",1,"Række-, kæde-, eller dobbelthus (lodret adskil...",1976,1,0,...,-,-,-,-,Bygning - stamdata,Bygningen har ikke afvigende etager,1,0,0,[]
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"{'buildingNumber': 1, 'buildingUsage': 'Dobbel...","{'buildingNumber': 1, 'buildingUsage': 'Dobbel...",1,Dobbelthus,1898,1,0,...,-,-,-,-,Bygning - stamdata,Bygningen har ikke afvigende etager,1,0,0,[]
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,"{'buildingNumber': 1, 'buildingUsage': 'Etageb...","{'buildingNumber': 1, 'buildingUsage': 'Etageb...",1,"Etagebolig-bygning, flerfamiliehus eller to-fa...",1917,5,0,...,(UDFASES) Bygningen har ingen supplerende varme,-,-,-,Bygning - stamdata,Bygningen har afvigende etager,173,0,0,[BRANDSIKRING 1917 FÆRDIGMELDT 050789]
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"{'buildingNumber': 1, 'buildingUsage': 'Række-...","{'buildingNumber': 1, 'buildingUsage': 'Række-...",1,"Række-, kæde- og klyngehus",1990,1,0,...,(UDFASES) Bygningen har ingen supplerende varme,-,-,-,Bygning - stamdata,Bygningen har ikke afvigende etager,1,0,0,[5 M2 UDHUS IKKE MEDREGNET I AREALET]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"{'buildingNumber': 1, 'buildingUsage': 'Fritli...","{'buildingNumber': 1, 'buildingUsage': 'Fritli...",1,Fritliggende enfamiliehus,1973,1,1998,...,-,-,-,-,Bygning - stamdata,Bygningen har ikke afvigende etager,1,0,0,[Stedfæstelse fra geokodning 2008]
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"{'buildingNumber': 1, 'buildingUsage': 'Fritli...","{'buildingNumber': 1, 'buildingUsage': 'Fritli...",1,Fritliggende enfamiliehus,1978,1,0,...,Brændeovne og lignende med skorsten,-,-,-,Bygning - stamdata,Bygningen har ikke afvigende etager,1,0,0,[]
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"{'buildingNumber': 1, 'buildingUsage': 'Fritli...","{'buildingNumber': 1, 'buildingUsage': 'Fritli...",1,Fritliggende enfamiliehus,1953,1,0,...,-,-,-,-,Bygning - stamdata,Bygningen har ikke afvigende etager,1,0,0,[]
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"{'buildingNumber': 4, 'buildingUsage': 'Fritli...","{'buildingNumber': 4, 'buildingUsage': 'Fritli...",4,Fritliggende enfamiliehus,1960,1,0,...,-,-,-,-,Bygning - stamdata,Bygningen har ikke afvigende etager,1,0,0,[]


In [8]:
########################################################################################################
# -------------------  save the result !       -------------------------
########################################################################################################
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\Bolgia_char_buildingInfo_EXPANDED.csv'
csv_reader.to_csv(path, encoding='utf-8')

# ----------------------- Expanding column: char_unitInfo
## i will take two columns guid, municipality code, property code and char_unitInfo - expand the bbrInfoBox column and save it as a seperated file

In [ ]:
import ast

#---------------------  read in the big file for only the key columns here
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'
csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['guid_code', 'property_esrCode','municipalityCode', 'char_unitInfo'])
csv_reader

In [8]:
# Convert strings to dictionaries
csv_reader['char_unitInfo_dic'] = csv_reader['char_unitInfo'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else {})

In [ ]:
# Normalize the dictionaries into separate columns
df_expanded = pd.json_normalize(csv_reader['char_unitInfo_dic'])

# Concatenate the expanded DataFrame with the original DataFrame
csv_reader = pd.concat([csv_reader, df_expanded], axis=1)
csv_reader

In [17]:
from collections import Counter
Counter(csv_reader['appliance'].tolist())

Counter({'Fritliggende enfamiliehus': 566699,
         'Bolig i etageejendom, flerfamiliehus eller to-familiehus': 283880,
         'Række-, kæde- eller dobbelthus (lodret adskillelse mellem enhederne).': 73467,
         'Stuehus til landbrugsejendom': 45113,
         'Række-, kæde- og klyngehus': 42327,
         'Dobbelthus': 15236,
         'Kollegiebolig': 4236,
         'Sammenbygget enfamiliehus': 637})

In [11]:
########################################################################################################
# -------------------  save the result !       -------------------------
########################################################################################################
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\Column_4_char_unitInfo_EXPANDED.csv'
csv_reader.to_csv(path, encoding='utf-8')

In [10]:
csv_reader

,guid_code,property_esrCode,municipalityCode,char_unitInfo,char_unitInfo_dic,appliance,propertyUnitType,propertyCondemned,areaUnit,areaLiving,...,toiletQuantity,bathroomCode,bathroomQuantity,kitchenCode,energyCode,heatingInstallationCode,heatingCode,heatingSecondaryCode,usageCode,notes
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"{'appliance': 'Række-, kæde- eller dobbelthus ...","{'appliance': 'Række-, kæde- eller dobbelthus ...","Række-, kæde- eller dobbelthus (lodret adskill...",Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,141,141,...,1,Badeværelse i enheden,1,Eget køkken med afløb,400 V el fra værk,-,-,-,-,[]
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"{'appliance': 'Række-, kæde- eller dobbelthus ...","{'appliance': 'Række-, kæde- eller dobbelthus ...","Række-, kæde- eller dobbelthus (lodret adskill...",Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,122,122,...,2,Badeværelse i enheden,2,Eget køkken med afløb,-,-,-,-,-,[]
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"{'appliance': 'Dobbelthus', 'propertyUnitType'...","{'appliance': 'Dobbelthus', 'propertyUnitType'...",Dobbelthus,Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,70,70,...,2,Badeværelse i enheden,2,Eget køkken med afløb,-,-,-,-,-,[]
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,"{'appliance': 'Bolig i etageejendom, flerfamil...","{'appliance': 'Bolig i etageejendom, flerfamil...","Bolig i etageejendom, flerfamiliehus eller to-...",Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,71,71,...,1,Badeværelse i enheden,1,Eget køkken med afløb,-,-,-,-,-,"[HERAF 13 KVM PÅ 6. SAL, Balkonareal på 13 kvm..."
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"{'appliance': 'Række-, kæde- og klyngehus', 'p...","{'appliance': 'Række-, kæde- og klyngehus', 'p...","Række-, kæde- og klyngehus",Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,86,86,...,1,Badeværelse i enheden,1,Eget køkken med afløb,400 V el fra værk,-,-,-,-,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"{'appliance': 'Fritliggende enfamiliehus', 'pr...","{'appliance': 'Fritliggende enfamiliehus', 'pr...",Fritliggende enfamiliehus,Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,246,246,...,2,Badeværelse i enheden,2,Eget køkken med afløb,-,-,-,-,-,[]
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"{'appliance': 'Fritliggende enfamiliehus', 'pr...","{'appliance': 'Fritliggende enfamiliehus', 'pr...",Fritliggende enfamiliehus,Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,144,144,...,1,Badeværelse i enheden,1,Eget køkken med afløb,400 V el fra værk,-,-,-,-,[]
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"{'appliance': 'Fritliggende enfamiliehus', 'pr...","{'appliance': 'Fritliggende enfamiliehus', 'pr...",Fritliggende enfamiliehus,Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,119,119,...,2,Badeværelse i enheden,1,Eget køkken med afløb,-,-,-,-,-,[]
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"{'appliance': 'Fritliggende enfamiliehus', 'pr...","{'appliance': 'Fritliggende enfamiliehus', 'pr...",Fritliggende enfamiliehus,Egentlig beboelseslejlighed,Ikke kondemneret boligenhed,106,106,...,1,Badeværelse i enheden,1,Eget køkken med afløb,-,-,-,-,-,[]


# ----------------------- Expanding column: char_lotInfo                 

## i will take two columns guid, municipality code, property code and char_unitInfo - expand the bbrInfoBox column and save it as a seperated file

In [ ]:
import ast

#---------------------  read in the big file for only the key columns here

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['guid_code', 'property_esrCode','municipalityCode', 'char_lotInfo'])
csv_reader

In [3]:
# Convert strings to dictionaries
csv_reader['char_lotInfo_dic'] = csv_reader['char_lotInfo'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else {})

In [5]:
# Normalize the dictionaries into separate columns
df_expanded = pd.json_normalize(csv_reader['char_lotInfo_dic'])

# Concatenate the expanded DataFrame with the original DataFrame
csv_reader = pd.concat([csv_reader, df_expanded], axis=1)
csv_reader

,guid_code,property_esrCode,municipalityCode,char_lotInfo,char_lotInfo_dic,drain,drainPermission,watersupplyCode,notes
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"{'drain': '5', 'drainPermission': '-', 'waters...","{'drain': '5', 'drainPermission': '-', 'waters...",5,-,Alment vandforsyningsanlæg,[]
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"{'drain': '10', 'drainPermission': '-', 'water...","{'drain': '10', 'drainPermission': '-', 'water...",10,-,Alment vandforsyningsanlæg,[]
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"{'drain': '10', 'drainPermission': '-', 'water...","{'drain': '10', 'drainPermission': '-', 'water...",10,-,Alment vandforsyningsanlæg,[]
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,NaN,{},NaN,NaN,NaN,NaN
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"{'drain': '10', 'drainPermission': '-', 'water...","{'drain': '10', 'drainPermission': '-', 'water...",10,-,Privat vandforsyningsanlæg,[SKÆVINGE VANDVÆRK]
...,...,...,...,...,...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"{'drain': '10', 'drainPermission': '-', 'water...","{'drain': '10', 'drainPermission': '-', 'water...",10,-,Privat vandforsyningsanlæg,[]
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"{'drain': '10', 'drainPermission': '-', 'water...","{'drain': '10', 'drainPermission': '-', 'water...",10,-,Alment vandforsyningsanlæg,[]
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"{'drain': '10', 'drainPermission': '-', 'water...","{'drain': '10', 'drainPermission': '-', 'water...",10,-,Alment vandforsyningsanlæg,[]
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"{'drain': '1', 'drainPermission': '-', 'waters...","{'drain': '1', 'drainPermission': '-', 'waters...",1,-,Privat vandforsyningsanlæg,[]


In [6]:
########################################################################################################
# -------------------  save the result !       -------------------------
########################################################################################################
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\Column_char_lotInfo_EXPANDED.csv'
csv_reader.to_csv(path, encoding='utf-8')

In [18]:
566699+283880+73467+42327+15236+637

982246

# ----------------------- Expanding column: char_cadastralInfos                 

## i will take two columns guid, municipality code, property code and char_unitInfo - expand the bbrInfoBox column and save it as a seperated file

In [12]:
import ast

#---------------------  read in the big file for only the key columns here

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'

csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['guid_code', 'property_esrCode','municipalityCode', 'char_cadastralInfos'])
csv_reader

,guid_code,property_esrCode,municipalityCode,char_cadastralInfos
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"[{'cadestralNumber': '58ab', 'area': 562, 'roa..."
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"[{'cadestralNumber': '4gd', 'area': 217, 'road..."
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"[{'cadestralNumber': '386', 'area': 343, 'road..."
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,[]
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"[{'cadestralNumber': '9nx', 'area': 103, 'road..."
...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"[{'cadestralNumber': '207', 'area': 1084, 'roa..."
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"[{'cadestralNumber': '11aø', 'area': 1023, 'ro..."
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"[{'cadestralNumber': '2fø', 'area': 750, 'road..."
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"[{'cadestralNumber': '2dp', 'area': 6735, 'roa..."


In [45]:
csv_reader

,guid_code,property_esrCode,municipalityCode,char_cadastralInfos,char_cadastralInfos_dic,char_cadastralInfos_TotalArea,char_cadastralInfos_NumberOfLandRegisters
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"[{'cadestralNumber': '58ab', 'area': 562, 'roa...","[{'cadestralNumber': '58ab', 'area': 562, 'roa...",0,0
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"[{'cadestralNumber': '4gd', 'area': 217, 'road...","[{'cadestralNumber': '4gd', 'area': 217, 'road...",0,0
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"[{'cadestralNumber': '386', 'area': 343, 'road...","[{'cadestralNumber': '386', 'area': 343, 'road...",0,0
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,[],[],0,0
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"[{'cadestralNumber': '9nx', 'area': 103, 'road...","[{'cadestralNumber': '9nx', 'area': 103, 'road...",0,0
...,...,...,...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"[{'cadestralNumber': '207', 'area': 1084, 'roa...","[{'cadestralNumber': '207', 'area': 1084, 'roa...",0,0
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"[{'cadestralNumber': '11aø', 'area': 1023, 'ro...","[{'cadestralNumber': '11aø', 'area': 1023, 'ro...",0,0
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"[{'cadestralNumber': '2fø', 'area': 750, 'road...","[{'cadestralNumber': '2fø', 'area': 750, 'road...",0,0
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"[{'cadestralNumber': '2dp', 'area': 6735, 'roa...","[{'cadestralNumber': '2dp', 'area': 6735, 'roa...",0,0


In [ ]:
# Convert strings to dictionaries
csv_reader['char_cadastralInfos_dic'] = csv_reader['char_cadastralInfos_dic_nolust'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else {})

In [34]:
# create empty column for  the total area 
csv_reader['char_cadastralInfos_TotalArea'] = 0

In [44]:
# create empty column for the number of land registers 
csv_reader['char_cadastralInfos_NumberOfLandRegisters'] = 0

In [ ]:
# row_counter = 0
for index,row in csv_reader.iterrows():
    row_counter +=1
    print(f'-------------------------------------- row number {row_counter} out of 1.031.595')
#     print(f'-------------------------------------- {row["guid_code"]}')
#     print()
    land_register_counter = 0
    area_sum = 0
    for object_dic in row['char_cadastralInfos_dic']:
#         area = object_dic['area']
        area_sum = object_dic['area']
        
        land_register_counter += 1
    
    # update area sum
    csv_reader.at[index, 'char_cadastralInfos_TotalArea'] = area_sum
    
    # update lande registers 
    csv_reader.at[index, 'char_cadastralInfos_NumberOfLandRegisters'] = land_register_counter
        
#     # print number of object
#     print(f'=====> object number: {object_counter}')

#     # print area
#     print(f'==========> Area : {area}')

#     #print area sum
#     print(f'==========> Area SUM ! : {area_sum}')
        

In [48]:
csv_reader

,guid_code,property_esrCode,municipalityCode,char_cadastralInfos,char_cadastralInfos_dic,char_cadastralInfos_TotalArea,char_cadastralInfos_NumberOfLandRegisters
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"[{'cadestralNumber': '58ab', 'area': 562, 'roa...","[{'cadestralNumber': '58ab', 'area': 562, 'roa...",562,1
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"[{'cadestralNumber': '4gd', 'area': 217, 'road...","[{'cadestralNumber': '4gd', 'area': 217, 'road...",217,1
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"[{'cadestralNumber': '386', 'area': 343, 'road...","[{'cadestralNumber': '386', 'area': 343, 'road...",343,1
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,[],[],0,0
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"[{'cadestralNumber': '9nx', 'area': 103, 'road...","[{'cadestralNumber': '9nx', 'area': 103, 'road...",103,1
...,...,...,...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"[{'cadestralNumber': '207', 'area': 1084, 'roa...","[{'cadestralNumber': '207', 'area': 1084, 'roa...",1084,1
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"[{'cadestralNumber': '11aø', 'area': 1023, 'ro...","[{'cadestralNumber': '11aø', 'area': 1023, 'ro...",1023,1
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"[{'cadestralNumber': '2fø', 'area': 750, 'road...","[{'cadestralNumber': '2fø', 'area': 750, 'road...",750,1
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"[{'cadestralNumber': '2dp', 'area': 6735, 'roa...","[{'cadestralNumber': '2dp', 'area': 6735, 'roa...",6735,1


In [49]:
########################################################################################################
# -------------------  save the result !       -------------------------
########################################################################################################
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\Column_5_char_cadastralInfos_EXPANDED.csv'
csv_reader.to_csv(path, encoding='utf-8')

# ----------------------- Expanding column: char_additionalBuldingInfos

## i will take two columns guid, municipality code, property code and char_unitInfo - expand the bbrInfoBox column and save it as a seperated file

In [4]:
import ast

#---------------------  read in the big file for only the key columns here

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'
csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['guid_code', 'property_esrCode','municipalityCode', 'char_additionalBuldingInfos'])
csv_reader

,guid_code,property_esrCode,municipalityCode,char_additionalBuldingInfos
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,"[{'buildingNumber': 3, 'buildingUsage': 'Carpo..."
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"[{'buildingNumber': 2, 'buildingUsage': 'Udhus..."
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"[{'buildingNumber': 3, 'buildingUsage': 'Udhus..."
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,[]
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,[]
...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"[{'buildingNumber': 2, 'buildingUsage': 'Carpo..."
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,"[{'buildingNumber': 3, 'buildingUsage': 'Udhus..."
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"[{'buildingNumber': 2, 'buildingUsage': 'Garag..."
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"[{'buildingNumber': 1, 'buildingUsage': 'Bygni..."


In [5]:
# Convert strings to dictionaries
csv_reader['char_additionalBuldingInfos_dic'] = csv_reader['char_additionalBuldingInfos'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else {})

In [42]:
######### variables that I will create 
# ----- udhus_dummy
# ----- number_of_udhus
# ----- carport_dummy
# ----- number_of_carports
# ----- garage_dummy
# ----- number_of_garage
# ----- greenhouse_dummy
# ----- number_of_GreenHouses
# ----- swimmingPool_dummy
# ----- number_of_SwimmngPools

csv_reader['udhus_dummy'] = ''
csv_reader['number_of_udhus'] = ''
csv_reader['carport_dummy'] = ''
csv_reader['number_of_carports'] = ''
csv_reader['garage_dummy'] = ''
csv_reader['number_of_garage'] = ''
csv_reader['greenhouse_dummy'] = ''
csv_reader['number_of_GreenHouses'] = ''
csv_reader['swimmingPool_dummy'] = ''
csv_reader['number_of_SwimmngPools'] = ''
csv_reader['mixed_property_dummy'] = ''
csv_reader['number_of_mixed_property'] = ''


In [ ]:
row_counter = 0
for index,row in csv_reader.iterrows():
    row_counter +=1
    print(f'-------------------------------------- row number {row_counter} out of 1.031.595')
    
    # ---------- setup variables
    var_udhus_dummy=0
    var_carport_dummy=0
    var_garage_dummy=0
    var_greenhouse_dummy=0
    var_swimmingPool_dummy=0
    var_mixed_property_dummy=0

    var_number_of_udhus = 0
    var_number_of_carports = 0
    var_number_of_garage = 0
    var_number_of_GreenHouses = 0
    var_number_of_SwimmngPools = 0
    var_number_of_mixed_property = 0
    
    for building in row['char_additionalBuldingInfos_dic']:
        building_usage= building['buildingUsage']
        
        if building_usage == "Udhus":
            var_number_of_udhus +=1
        if building_usage == "Carport":
            var_number_of_carports +=1
        if building_usage == "Garage":
            var_number_of_garage +=1
        if building_usage == "Drivhus":
            var_number_of_GreenHouses +=1
        if building_usage == "Svømmehal":
            var_number_of_SwimmngPools+=1
        else:
            var_number_of_mixed_property +=1
    
    if var_number_of_udhus > 0:
        var_udhus_dummy = 1
        
    if var_number_of_carports > 0:
        var_carport_dummy = 1
    
    if var_number_of_garage > 0:
        var_garage_dummy = 1
    
    if var_number_of_GreenHouses > 0:
        var_greenhouse_dummy = 1
    
    if var_number_of_SwimmngPools > 0:
        var_swimmingPool_dummy = 1
    
    if var_number_of_mixed_property > 0:
        var_mixed_property_dummy = 1

    
    # ------------------ update columns with the result
    
    # update udhus dummy
    csv_reader.at[index,'udhus_dummy'] =var_udhus_dummy
    # update carport dummy
    csv_reader.at[index, 'carport_dummy'] =var_carport_dummy
    # update garage dummy
    csv_reader.at[index, 'garage_dummy'] =var_garage_dummy
    # update greenhouse dummy
    csv_reader.at[index,'greenhouse_dummy'] =var_greenhouse_dummy
    # update swimmingpool dummy
    csv_reader.at[index, 'swimmingPool_dummy'] =var_swimmingPool_dummy
    # update mixed property dummy
    csv_reader.at[index, 'mixed_property_dummy'] =var_mixed_property_dummy
    # update udhus number
    csv_reader.at[index,'number_of_udhus'] =var_number_of_udhus
    # update carports number
    csv_reader.at[index,'number_of_carports'] =var_number_of_carports
    # update garage number
    csv_reader.at[index,'number_of_garage'] =var_number_of_garage
    # update greenhouse number
    csv_reader.at[index,'number_of_GreenHouses'] =var_number_of_GreenHouses
    # update swimmingpool number
    csv_reader.at[index,'number_of_SwimmngPools'] =var_number_of_SwimmngPools
    # update mixed property number
    csv_reader.at[index,'number_of_mixed_property'] =var_number_of_mixed_property
    


In [53]:
########################################################################################################
# -------------------  save the result !       -------------------------
########################################################################################################
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\Column_6_char_additionalBuldingInfos_EXPANDED.csv'
csv_reader.to_csv(path, encoding='utf-8')

# ****** wait with char_floorInfos and char_additional_florsinfos *******

# ----------------------- Expanding column: char_floorInfos

## i will take two columns guid, municipality code, property code and char_floorInfos - expand the bbrInfoBox column and save it as a seperated file

In [2]:
import ast

#---------------------  read in the big file for only the key columns here -------------------
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_5_Expand_columns_and_filter\part_1_Boliga_nameChanged_columnsDeleted.csv'
csv_reader  = pd.read_csv(path, encoding='utf-8', low_memory=False, usecols=['guid_code', 'property_esrCode','municipalityCode', 'char_floorInfos'])
csv_reader

,guid_code,property_esrCode,municipalityCode,char_floorInfos
0,9133416f-191b-496e-88bb-62b3a46a370a,981498,751,[]
1,b1a52c68-cc3c-4ee0-b794-d9e638cc26e4,134998,265,"[{'designation': 'KL', 'area': 36, 'usedAtticA..."
2,f45b8fc3-af55-4907-b05e-bbc3f98c5935,4216,376,"[{'designation': '1', 'area': 28, 'usedAtticAr..."
3,0d3618c0-35a1-49bc-b2cb-9228ef5065f9,496120,101,[]
4,e824b5cd-d1ae-4599-9b1d-7bff5cc52255,153129,219,"[{'designation': '1', 'area': 41, 'usedAtticAr..."
...,...,...,...,...
1031590,bef171e1-3110-4be3-8553-f5e00197d95e,13570,580,"[{'designation': '1', 'area': 86, 'usedAtticAr..."
1031591,ce1f5b0b-053a-4661-9cb2-632d1a182d5f,141287,779,[]
1031592,845ce2df-d0ca-45cf-b6ca-df6ca620a685,4809,810,"[{'designation': '1', 'area': 36, 'usedAtticAr..."
1031593,81ce9112-1089-4179-a38e-e89dd148faac,128600,615,"[{'designation': 'KL', 'area': 40, 'usedAtticA..."


In [3]:
# Convert strings to dictionaries
csv_reader['char_floorInfos_dic'] = csv_reader['char_floorInfos'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else {})

In [ ]:
csv_reader

In [27]:
display(csv_reader['char_floorInfos'][1500])

display(csv_reader['char_floorInfos_dic'][1500])

"[{'designation': 'KL', 'area': 68, 'usedAtticArea': 0, 'basementLivingArea': 68, 'highBasementArea': 0, 'accessArea': 0, 'notes': []}, {'designation': '1', 'area': 55, 'usedAtticArea': 55, 'basementLivingArea': 55, 'highBasementArea': 0, 'accessArea': 0, 'notes': []}]"

[{'designation': 'KL',
  'area': 68,
  'usedAtticArea': 0,
  'basementLivingArea': 68,
  'highBasementArea': 0,
  'accessArea': 0,
  'notes': []},
 {'designation': '1',
  'area': 55,
  'usedAtticArea': 55,
  'basementLivingArea': 55,
  'highBasementArea': 0,
  'accessArea': 0,
  'notes': []}]

In [24]:
for i in csv_reader['char_floorInfos_dic'][1500]:
    print(i)

{'designation': 'KL', 'area': 68, 'usedAtticArea': 0, 'basementLivingArea': 68, 'highBasementArea': 0, 'accessArea': 0, 'notes': []}
{'designation': '1', 'area': 55, 'usedAtticArea': 55, 'basementLivingArea': 55, 'highBasementArea': 0, 'accessArea': 0, 'notes': []}


In [ ]:
number_of_floors = []
designation_list = []

count = 0 
for index,row in csv_reader.iterrows():
    count +=1
    print(f'------------------------------------- {count} out of 1.031.595')
#     print('======= FULL SETUP')
#     display(row['char_floorInfos_dic'])
    
#     print(f'-------------- len : {len(row["char_floorInfos_dic"])}')
    number_of_floors.append(len(row["char_floorInfos_dic"]))
    
#     print('======= SEPERATE')
    for dic in row['char_floorInfos_dic']:
        designation_list.append(dic['designation'])
#         display(dic)
#         print()
        
        

#         for floor in object_row:
#             print()
#             count +=1
#         print(f'------------{count}')
#         floors_list.append(count)
        
        

In [36]:
display(set(number_of_floors))
print()
display(set(designation_list))


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19}

{'',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '3',
 '4',
 '5',
 '6',
 '65',
 '7',
 '8',
 '9',
 'K2',
 'K3',
 'KL'}

In [ ]:
------- 22
[{'designation': 'KL',
  'area': 53,
  'usedAtticArea': 0,
  'basementLivingArea': 53,
  'highBasementArea': 53,
  'accessArea': 0,
  'notes': []},
 {'designation': '1',
  'area': 120,
  'usedAtticArea': 120,
  'basementLivingArea': 120,
  'highBasementArea': 0,
  'accessArea': 0,
  'notes': []}]